### Photon Compatibility Analysis
This cell analyzes the `sql_df` DataFrame to identify how much of a Spark application's workload can be accelerated by Photon.
It parses the JSON query plans, flags compatible operations, and calculates a "Photon Compatibility Score" for each application.
A higher score indicates a greater portion of the application's SQL workload is Photon-compatible, suggesting better performance. A low score highlights opportunities for optimization by refactoring non-compatible operations.

In [0]:
 %sql
-- diff of more than 2x btwn min and median or median and max
-- any spill
-- chunk sizes
-- diff of more than 2x processing rate
-- with task completion rate

with rawmetrics as (
select *, try_variant_get(parse_json(raw_json),'$.duration', 'array<double>') as duration, try_variant_get(parse_json(raw_json),'$.memoryBytesSpilled', 'array<double>') as memoryBytesSpilled, try_variant_get(parse_json(raw_json),'$.diskBytesSpilled', 'array<double>') as diskBytesSpilled, try_variant_get(parse_json(raw_json),'$.inputMetrics.bytesRead', 'array<double>') as bytesRead , 
try_variant_get(parse_json(raw_json),'$.outputMetrics.bytesWritten', 'array<double>') as bytesWritten , try_variant_get(parse_json(raw_json),'$.shuffleReadMetrics.readBytes', 'array<double>') readBytes,
try_variant_get(parse_json(raw_json),'$.shuffleReadMetrics.fetchWaitTime', 'array<double>') fetchWaitTime,
try_variant_get(parse_json(raw_json),'$.shuffleWriteMetrics.writeBytes', 'array<double>') writeBytes
from parquet.`s3://oetrta/robertostream/sparkmetrics/onektest/2025-12-04/task_summaries`),

polmetrics as (select *, duration[0] as min, duration[2] as median, duration[4] as maxim, memoryBytesSpilled[2] as spillmed, memoryBytesSpilled[4] as spillmax, diskBytesSpilled[2] as diskspillmed, diskBytesSpilled[4] as diskspillmax, bytesRead[0] as minbytes, bytesRead[2] as medbytes, bytesRead[4] as maxbytes, bytesWritten[0] as minbyteswr, bytesWritten[2] as medbyteswr, bytesWritten[4] as maxbyteswr, readbytes[0] as minsr, readbytes[2] as medsr, readbytes[4] as maxsr, writebytes[0] as minsw, writebytes[2] as medsw, writebytes[4] as maxsw,
map('minbytes', minbytes, 'minbyteswr', minbyteswr, 'minsr', minsr, 'minsw',minsw) as minchunks,
map('minbytes', medbytes, 'medbyteswr', medbyteswr, 'medsr', medsr, 'medsw',medsw) as medchunks,
map('maxbytes', maxbytes, 'maxbyteswr', maxbyteswr, 'maxsr', maxsr, 'maxsw',maxsw) as maxchunks,
map('minbrpr', try_divide(minbytes, min), 'minbwpr', try_divide(minbyteswr, min), 'minsrpr', try_divide(minsr, min), 'minswpr', try_divide(minsw, min)) as minpr,
map('medbrpr', try_divide(medbytes, median), 'medbwpr', try_divide(medbyteswr, median), 'medsrpr', try_divide(medsr, median), 'medswpr', try_divide(medsw, median)) as medpr,
map('maxbrpr', try_divide(maxbytes, maxim), 'maxbwpr', try_divide(maxbytes, maxim), 'maxsrpr', try_divide(maxsr, maxim), 'maxswpr', try_divide(maxsw, maxim)) as maxpr
from rawmetrics),

mapping as (select *, diskspillmed, diskspillmax,  map_filter(minchunks, (k, v) -> v > 512000000) as minchunkmap, map_filter(minchunks, (k, v) -> v > 512000000) as medchunkmap, map_filter(minchunks, (k, v) -> v > 512000000) as maxchunkmap,
map_filter(minpr, (k, v) -> v between 0.01 and  10000) as minprmap, map_filter(medpr, (k, v) -> v between 0.01 and 10000) as medprmap, map_filter(maxpr, (k, v) -> v between 0.01 and 10000) as maxprmap
from polmetrics),

precheck as (select cluster_id, cluster_name, stage_id, diskspillmed, diskspillmax, min, median, maxim, spillmed, spillmax, cardinality(minchunkmap) as minchunks, cardinality(medchunkmap) as medchunks, cardinality(maxchunkmap) as maxchunks,
cardinality(minprmap) as minpr, cardinality(medprmap) as medpr, cardinality(maxprmap) as maxpr
from mapping),

goldcheck as (select cluster_id, cluster_name, stage_id, CASE WHEN (median > (2 * min) OR maxim > (2 * median))  THEN 1 ELSE 0 END as skewbinary,
CASE WHEN (spillmed > 256000000 or spillmax > 256000000) THEN 1 ELSE 0 END as spillbinary,
CASE WHEN (diskspillmed > 256000000 or diskspillmax > 256000000) THEN 1 ELSE 0 END as diskspillbinary,
CASE WHEN (minchunks > 0 or medchunks > 0 or maxchunks > 0) THEN 1 ELSE 0 END as chunkbinary,
CASE WHEN (minpr > 0 or medpr > 0 or maxpr > 0) THEN 1 ELSE 0 END as prbinary
from precheck),

enrichjoin as (
  select goldcheck.*, cardinality(map_filter(map('skewbinary', skewbinary, 'spillbinary', spillbinary, 'diskspillbinary', diskspillbinary, 'chunkbinary', chunkbinary, 'trbinary', case when stages.task_completion_rate < 0.9 THEN 1 ELSE 0 END ), (k, v) -> v > 0)) as goldcheckfilter, stages.task_completion_rate, timestampdiff(millisecond, stages.submission_time, stages.completion_time) as duration
  from goldcheck
  join parquet.`s3://oetrta/robertostream/sparkmetrics/onektest/2025-12-04/stages`as stages
  on goldcheck.cluster_id = stages.cluster_id
  and goldcheck.stage_id = stages.stage_id
),

pu as (select cluster_id, cluster_name, stage_id, sum(case when goldcheckfilter > 0 then duration else 0 end) as ineffstag,
sum(duration) as stageagg
from enrichjoin
group by all),

finalsa as (select cluster_id, cluster_name, try_divide(sum(ineffstag), sum(stageagg)) * 100 as ineffstageagg
from pu
group by all)

select * from finalsa
where ineffstageagg > 25
--order by 3 desc, 4 desc

cluster_id,cluster_name,ineffstageagg
1124-170721-a9a1ujo2,job-1002150808513415-run-665157951722153-pipeline_cluster,29.325806709904718
1105-094830-5052vpci,job-1013043173213604-run-352537069257834-job_cluster,33.5124747573448
1201-121324-owiqe2vm,job-1010448409020169-run-505857690080789-wine_classifier_cluster,31.934877896055102
1105-093837-q43xt15e,job-1013043173213604-run-18317235773159-job_cluster,43.919526627218936
1107-063957-ovee7o5u,job-29338254504376-run-578790984329133-Job_cluster,75.38378991234971
1130-063957-7znxogsy,job-29338254504376-run-912789945208208-Job_cluster,85.00589679406951
1113-063957-bknbl3yq,job-29338254504376-run-285434227270289-Job_cluster,72.51582676473352
1123-063956-39wnogeq,job-29338254504376-run-625040118384747-Job_cluster,83.70835049216164
1110-063957-uc6t4s2a,job-29338254504376-run-664082146756887-Job_cluster,57.63469488973202
1122-063958-54v8lkxu,job-29338254504376-run-828696742087665-Job_cluster,76.57560459875293


In [0]:
 %sql
-- diff of more than 2x btwn min and median or median and max
-- any spill
-- chunk sizes
-- diff of more than 2x processing rate
-- with task completion rate

with rawmetrics as (
select *, try_variant_get(parse_json(raw_json),'$.duration', 'array<double>') as duration, try_variant_get(parse_json(raw_json),'$.memoryBytesSpilled', 'array<double>') as memoryBytesSpilled, try_variant_get(parse_json(raw_json),'$.diskBytesSpilled', 'array<double>') as diskBytesSpilled, try_variant_get(parse_json(raw_json),'$.inputMetrics.bytesRead', 'array<double>') as bytesRead , 
try_variant_get(parse_json(raw_json),'$.outputMetrics.bytesWritten', 'array<double>') as bytesWritten , try_variant_get(parse_json(raw_json),'$.shuffleReadMetrics.readBytes', 'array<double>') readBytes,
try_variant_get(parse_json(raw_json),'$.shuffleReadMetrics.fetchWaitTime', 'array<double>') fetchWaitTime,
try_variant_get(parse_json(raw_json),'$.shuffleWriteMetrics.writeBytes', 'array<double>') writeBytes
from parquet.`s3://oetrta/robertostream/sparkmetrics/onektest/2025-12-04/task_summaries`),

polmetrics as (select *, duration[0] as min, duration[2] as median, duration[4] as maxim, memoryBytesSpilled[2] as spillmed, memoryBytesSpilled[4] as spillmax, diskBytesSpilled[2] as diskspillmed, diskBytesSpilled[4] as diskspillmax, bytesRead[0] as minbytes, bytesRead[2] as medbytes, bytesRead[4] as maxbytes, bytesWritten[0] as minbyteswr, bytesWritten[2] as medbyteswr, bytesWritten[4] as maxbyteswr, readbytes[0] as minsr, readbytes[2] as medsr, readbytes[4] as maxsr, writebytes[0] as minsw, writebytes[2] as medsw, writebytes[4] as maxsw,
map('minbytes', minbytes, 'minbyteswr', minbyteswr, 'minsr', minsr, 'minsw',minsw) as minchunks,
map('minbytes', medbytes, 'medbyteswr', medbyteswr, 'medsr', medsr, 'medsw',medsw) as medchunks,
map('maxbytes', maxbytes, 'maxbyteswr', maxbyteswr, 'maxsr', maxsr, 'maxsw',maxsw) as maxchunks,
map('minbrpr', try_divide(minbytes, min), 'minbwpr', try_divide(minbyteswr, min), 'minsrpr', try_divide(minsr, min), 'minswpr', try_divide(minsw, min)) as minpr,
map('medbrpr', try_divide(medbytes, median), 'medbwpr', try_divide(medbyteswr, median), 'medsrpr', try_divide(medsr, median), 'medswpr', try_divide(medsw, median)) as medpr,
map('maxbrpr', try_divide(maxbytes, maxim), 'maxbwpr', try_divide(maxbytes, maxim), 'maxsrpr', try_divide(maxsr, maxim), 'maxswpr', try_divide(maxsw, maxim)) as maxpr
from rawmetrics),

mapping as (select *, diskspillmed, diskspillmax,  map_filter(minchunks, (k, v) -> v > 512000000) as minchunkmap, map_filter(minchunks, (k, v) -> v > 512000000) as medchunkmap, map_filter(minchunks, (k, v) -> v > 512000000) as maxchunkmap,
map_filter(minpr, (k, v) -> v between 0.01 and  10000) as minprmap, map_filter(medpr, (k, v) -> v between 0.01 and 10000) as medprmap, map_filter(maxpr, (k, v) -> v between 0.01 and 10000) as maxprmap
from polmetrics),

precheck as (select cluster_id, cluster_name, stage_id, diskspillmed, diskspillmax, min, median, maxim, spillmed, spillmax, cardinality(minchunkmap) as minchunks, cardinality(medchunkmap) as medchunks, cardinality(maxchunkmap) as maxchunks,
cardinality(minprmap) as minpr, cardinality(medprmap) as medpr, cardinality(maxprmap) as maxpr
from mapping),

goldcheck as (select cluster_id, cluster_name, stage_id, CASE WHEN (median > (2 * min) OR maxim > (2 * median))  THEN 1 ELSE 0 END as skewbinary,
CASE WHEN (spillmed > 256000000 or spillmax > 256000000) THEN 1 ELSE 0 END as spillbinary,
CASE WHEN (diskspillmed > 256000000 or diskspillmax > 256000000) THEN 1 ELSE 0 END as diskspillbinary,
CASE WHEN (minchunks > 0 or medchunks > 0 or maxchunks > 0) THEN 1 ELSE 0 END as chunkbinary,
CASE WHEN (minpr > 0 or medpr > 0 or maxpr > 0) THEN 1 ELSE 0 END as prbinary
from precheck),

enrichjoin as (
  select goldcheck.*, cardinality(map_filter(map('skewbinary', skewbinary, 'spillbinary', spillbinary, 'diskspillbinary', diskspillbinary, 'chunkbinary', chunkbinary, 'trbinary', case when stages.task_completion_rate < 0.9 THEN 1 ELSE 0 END ), (k, v) -> v > 0)) as goldcheckfilter, stages.stage_id, stages.task_completion_rate, timestampdiff(millisecond, stages.submission_time, stages.completion_time) as duration
  from goldcheck
  join parquet.`s3://oetrta/robertostream/sparkmetrics/onektest/2025-12-04/stages`as stages
  on goldcheck.cluster_id = stages.cluster_id
  and goldcheck.stage_id = stages.stage_id
)

select * from enrichjoin
where cluster_id = '1108-063956-zfgrta8i'
--where ineffstageagg > 25
--order by 3 desc, 4 desc

cluster_id,cluster_name,stage_id,skewbinary,spillbinary,diskspillbinary,chunkbinary,prbinary,goldcheckfilter,stage_id,task_completion_rate,duration
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,100,0,0,0,0,1,0,100,100.0,231
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,99,1,0,0,0,1,1,99,100.0,5596
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,93,0,0,0,0,1,0,93,100.0,1244
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,90,0,0,0,0,1,0,90,100.0,2209
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,88,1,0,0,0,1,1,88,100.0,180477
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,87,1,0,0,0,0,1,87,100.0,12055
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,86,1,0,0,0,1,1,86,100.0,2820
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,85,1,0,0,0,1,1,85,100.0,16613
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,84,0,0,0,0,1,0,84,100.0,50
1108-063956-zfgrta8i,job-29338254504376-run-27017753663878-Job_cluster,83,0,0,0,0,1,0,83,100.0,252
